In [2]:
from utils import time_op
import impmeas as imp
from tabulate import tabulate

solver = imp.GPMC()
imp.set_pmc_solver(solver)
vars = [f"x{idx+1}" for idx in range(10)] # we don't need more than that
imp.buddy_initialize(vars, nodenum=1<<20, cachesize=1<<15)

In [3]:
# tests the model counting capabilities of GPMC and BuDDy 
_, formula = time_op(imp.random_k_cnf)(10,45,7)

f = time_op(imp.Formula.parse)(formula)
satcount_gpmc = time_op(solver.satcount)(f.tseitin()[0])
print(f"GPMC satcount = {satcount_gpmc}")

f = time_op(imp.BuddyNode.parse)(formula)
satcount_bdd = int(time_op(f.expectation)()*2**len(vars))
print(f"BDD satcount = {satcount_bdd}")

[00000.5589 ms / 0.0006 s / 0.0000 min] random_k_cnf
[00001.2643 ms / 0.0013 s / 0.0000 min] parse
[00014.0970 ms / 0.0141 s / 0.0002 min] satcount
GPMC satcount = 654
[00001.6325 ms / 0.0016 s / 0.0000 min] parse
[00000.0093 ms / 0.0000 s / 0.0000 min] expectation
BDD satcount = 654


In [4]:
# computes the modified Blame value via the proposed BDD approach
f = time_op(imp.BuddyNode.parse)(formula)
print(f"bdd size = {f.nodecount}")
blame = time_op(imp.blame)(f, "x1", rho=lambda x: 1/(x+1), cutoff=1e-4, modified=True, debug=True)
print(f"B^rhofrac_x1(f) = {blame:.5f}")

[00001.9281 ms / 0.0019 s / 0.0000 min] parse
bdd size = 88
=== COMPUTING BLAME for x1 in <impmeas.formulas.buddy.BuddyNode object at 0x7f2ea44524a0> ===
k=0 d result=0.2617 max increase possible=0.3691 
k=1 d result=0.2930 max increase possible=0.0508 
k=2 d result=0.0508 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff=0.0001.
current value: 0.6055, can only be increased by 0.0000.
=== DONE ===
[00001.0931 ms / 0.0011 s / 0.0000 min] blame
B^rhofrac_x1(f) = 0.60547


In [5]:
# computes the modified Blame value via the proposed PMC approach
f = time_op(imp.Formula.parse)(formula)
blame = time_op(imp.blame)(f, "x1", rho=lambda x: 1/(x+1), cutoff=1e-4, modified=True, debug=True)
print(f"B^rhofrac_x1(f) = {blame:.5f}")

[00001.4510 ms / 0.0015 s / 0.0000 min] parse
=== COMPUTING BLAME for x1 in Formula with size 1734 ===
k=0 size of cnf: 1533 d result=0.2617 max increase possible=0.3691 
k=1 size of cnf: 1533 d result=0.2930 max increase possible=0.0508 
k=2 size of cnf: 1533 d result=0.0508 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff=0.0001.
current value: 0.6055, can only be increased by 0.0000.
=== DONE ===
[00151.7844 ms / 0.1518 s / 0.0025 min] blame
B^rhofrac_x1(f) = 0.60547


In [6]:
# We can see that Table-based approaches work also for smaller formulas
# generating random CNF
cnf, formula = imp.random_k_cnf(4,9,4)
f = imp.Table.parse(formula)
print("f =", f)
print("-"*30)

# computing Blame via Table based approach
blame = imp.blame(f, "x1", rho=lambda x: 1/(x+1), cutoff=1e-4, debug=False)
print(f"B^rhofrac_x1(f) = {blame:.5f}")

# computing Influence via Table based approach
infl = imp.influence(f, "x1")
print(f"I_x1(f) = {infl:.5f}")

# computing composition of Banzhaf value + HKR's quadratic CGM
hkr = imp.banzhaf(imp.hkr_cgm(f),"x1")
print(f"Bz_x1(E^kappaquad_f) = {hkr:.5f}")

# computing dominating and rectifying CGMs
dom = imp.dominating_cgm(f)
rec = imp.rectifying_cgm(f)
print("omega_f =", dom)
print("nu_f =", rec)

f = x2'x1'x4' | x2x4 | x1'x3 | x2x3
------------------------------
B^rhofrac_x1(f) = 0.52083
I_x1(f) = 0.37500
Bz_x1(E^kappaquad_f) = 0.24023
omega_f = x1x3 | x4x2 | x3x2
nu_f = x1x4 | x1x3 | x1x2 | x4x2 | x3x2


In [7]:
# Blame: The importance of variables in modules can actually increase (for non-monotonic modules).
# here, note that f = (x|y)^z is modular in h = x|y. However, x is considered as more relevant for f than for h:
h = imp.Table.parse("x|y")
f = h ^ imp.Table.parse("z")
print(f"B^rho_x(h):\t {imp.blame(h, 'x'):.4f}")
print(f"B^rho_x(f):\t {imp.blame(f, 'x'):.5f}")

B^rho_x(h):	 0.6250
B^rho_x(f):	 0.70833


In [8]:
# Blame: Counterexample for Property RANK
g = imp.Table.parse("~x1 & ~x0 & ~x2 | x1 & x0 & x2 | x3 & ~x0 & ~x2 | x3 & x0 & x2 | x3 & x1")
f = g | imp.Table.parse("z")
b = lambda f,x: imp.blame(f,x,rho=lambda x:2**-x)
print("g =", g)
print("f =", f)
print(f"B^rhoexp_(.)(g):\t {imp.ranking_from_val(b, g)}")
print(f"B^rhoexp_(.)(f):\t {imp.ranking_from_val(b, f)}")

print("-"*100)

g = imp.Table.parse("x1 & ~x0 & ~x2 | ~x1 & x0 | x3")
f = g | imp.Table.parse("z")    
b = lambda f,x: imp.blame(f,x,rho=lambda x:1/(x+1))
print("g =", g)
print("f =", f)
print(f"B^rhofrac_(.)(g):\t {imp.ranking_from_val(b, g)}")
print(f"B^rhofrac_(.)(f):\t {imp.ranking_from_val(b, f)}")

g = x2'x1'x0' | x2x1x0 | x2'x0'x3 | x2x0x3 | x1x3
f = x2'x1'x0' | x2x1x0 | x2'x0'x3 | x2x0x3 | x1x3 | z
B^rhoexp_(.)(g):	 x3:0.6250 < x0:0.7188 == x2:0.7188 < x1:0.7266
B^rhoexp_(.)(f):	 x3:0.4062 < x1:0.4980 < x0:0.5000 == x2:0.5000 < z:0.6172
----------------------------------------------------------------------------------------------------
g = x2'x1x0' | x1'x0 | x3
f = x2'x1x0' | x1'x0 | x3 | z
B^rhofrac_(.)(g):	 x2:0.2969 < x1:0.6302 == x0:0.6302 < x3:0.7188
B^rhofrac_(.)(f):	 x2:0.2250 < x3:0.4688 == z:0.4688 < x1:0.4802 == x0:0.4802


In [9]:
# Bz + dominating CGM: non-compliance to unbiasedness
f = imp.Table.parse("x|(y^z)")
bz_dcgm = lambda f,x: imp.banzhaf(imp.dominating_cgm(f),x)
print(f"(Bz*omega)_(.)(f):\t {imp.ranking_from_val(bz_dcgm, f)}")
print(f"(Bz*omega)_(.)(~f):\t {imp.ranking_from_val(bz_dcgm, ~f)}")

(Bz*omega)_(.)(f):	 y:0.2500 == z:0.2500 < x:0.7500
(Bz*omega)_(.)(~f):	 x:0.2500 == y:0.2500 == z:0.2500


In [10]:
# Table in the proof that shows limitations for the Blame
f1 = imp.Table.parse("(x^y) | z")
f2 = imp.Table.parse("(x | y | z)")
f3 = imp.Table.parse("x | y")
f4 = imp.Table.parse("x^y")
f = imp.Table.parse("x&y | ~x&z")
h = imp.Table.parse("x&(y|z) | ~x&z&y")
funcs = [f1, f2, f3, f4, f, h]

table = ["u(x) u(y) u(z) f1(u) f2(u) f3(u) f4(u) f(u) h(u) scs^u_x(f1) scs^u_x(f2) scs^u_x(f3) scs^u_x(f4) scs^u_x(f) scs^u_x(h)".split()]
for u in imp.iter_assignments(["x", "y", "z"]):
    table.append([ f"{v}/{int(u[v])}" for v in "xyz" ]
        + [ int(it[u]) for it in funcs ] 
        + [ imp.scs(it,"x",u) for it in funcs ])

print(tabulate(table, headers="firstrow", tablefmt="plain")) 


u(x)    u(y)    u(z)      f1(u)    f2(u)    f3(u)    f4(u)    f(u)    h(u)    scs^u_x(f1)    scs^u_x(f2)    scs^u_x(f3)    scs^u_x(f4)    scs^u_x(f)    scs^u_x(h)
x/0     y/0     z/0           0        0        0        0       0       0              0              0              0              0             1             1
x/0     y/0     z/1           1        1        0        0       1       0              2            inf              0              0             0             0
x/0     y/1     z/0           1        1        1        1       0       0              0            inf            inf              0             0             0
x/0     y/1     z/1           1        1        1        1       1       1              1            inf            inf              0             1           inf
x/1     y/0     z/0           1        1        1        1       0       0              0              0              0              0             1           inf
x/1     y/0     z/1   

In [17]:
# prints tables with Boolean functions and corresponding importance rankings induced by Blame, Influence, etc
fs = [ 
    "x|y|z",
    "x&(~y)&z",
    "x&(~y)",
    "x^y^z",
    "x|(y^z)",
    "x|(y&z)",
]

v_qh = lambda f,x: imp.banzhaf(imp.hkr_cgm(f),x)
v_dcgm = lambda f,x: imp.banzhaf(imp.dominating_cgm(f),x)
v_rcgm = lambda f,x: imp.banzhaf(imp.rectifying_cgm(f),x)
blame = imp.blame

for v, name in [(blame, "B^rhofrac"), (v_dcgm, "Bz*omega"), (v_rcgm, "Bz*nu"), (v_qh, "Bz*E")]:
    valnames = [f"{name}_{var}(f)" for var in ["x", "y", "z"]]
    header = ["f"] + valnames
    table = []
    for f in fs:
        f_binfunc = imp.Table.parse(f)
        row = [ f ]
        for var in ["x", "y", "z"]: 
            b = v(f_binfunc, var)
            row.append(b)
        table.append(row)
    print("="*100)
    print( tabulate(table, header, tablefmt="plain") )

f           B^rhofrac_x(f)    B^rhofrac_y(f)    B^rhofrac_z(f)
x|y|z             0.416667          0.416667          0.416667
x&(~y)&z          0.416667          0.416667          0.416667
x&(~y)            0.625             0.625             0
x^y^z             1                 1                 1
x|(y^z)           0.625             0.708333          0.708333
x|(y&z)           0.8125            0.416667          0.416667
f           Bz*omega_x(f)    Bz*omega_y(f)    Bz*omega_z(f)
x|y|z                0.25             0.25             0.25
x&(~y)&z             0.25             0.25             0.25
x&(~y)               0.5              0.5              0
x^y^z                0.25             0.25             0.25
x|(y^z)              0.75             0.25             0.25
x|(y&z)              0.75             0.25             0.25
f           Bz*nu_x(f)    Bz*nu_y(f)    Bz*nu_z(f)
x|y|z             0.25          0.25          0.25
x&(~y)&z          0.25          0.25          0.25
x&(